In [1]:
import pandas as pd
from scipy import stats
import datetime as dt

In [2]:
retail_data = pd.read_csv(r'F:\Development\Data Glacier Internship\Week7\forecasting_case_study.csv', low_memory=False)

In [3]:
retail_data_copy = retail_data.copy()

In [4]:
retail_data_copy.head()

,Product,date,Sales,Price Discount (%),In-Store Promo,Catalogue Promo,Store End Promo,Google_Mobility,Covid_Flag,V_DAY,EASTER,CHRISTMAS
0,SKU1,2/5/2017,27750,0%,0,0,0,0.0,0,0,0,0
1,SKU1,2/12/2017,29023,0%,1,0,1,0.0,0,1,0,0
2,SKU1,2/19/2017,45630,17%,0,0,0,0.0,0,0,0,0
3,SKU1,2/26/2017,26789,0%,1,0,1,0.0,0,0,0,0
4,SKU1,3/5/2017,41999,17%,0,0,0,0.0,0,0,0,0


In [5]:
retail_data_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Product             1218 non-null   object 
 1   date                1218 non-null   object 
 2   Sales               1218 non-null   int64  
 3   Price Discount (%)  1218 non-null   object 
 4   In-Store Promo      1218 non-null   int64  
 5   Catalogue Promo     1218 non-null   int64  
 6   Store End Promo     1218 non-null   int64  
 7   Google_Mobility     1218 non-null   float64
 8   Covid_Flag          1218 non-null   int64  
 9   V_DAY               1218 non-null   int64  
 10  EASTER              1218 non-null   int64  
 11  CHRISTMAS           1218 non-null   int64  
dtypes: float64(1), int64(8), object(3)
memory usage: 114.3+ KB


In [6]:
retail_data_copy.describe()

,Sales,In-Store Promo,Catalogue Promo,Store End Promo,Google_Mobility,Covid_Flag,V_DAY,EASTER,CHRISTMAS
count,1218.000000,1218.000000,1218.000000,1218.000000,1218.000000,1218.000000,1218.000000,1218.000000,1218.000000
mean,30294.678982,0.472085,0.212644,0.348933,-2.377406,0.226601,0.019704,0.019704,0.018883
std,35032.527297,0.499425,0.409346,0.476828,5.806291,0.418804,0.139040,0.139040,0.136169
min,0.000000,0.000000,0.000000,0.000000,-28.490000,0.000000,0.000000,0.000000,0.000000
25%,7212.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,19742.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,40282.250000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,288322.000000,1.000000,1.000000,1.000000,3.900000,1.000000,1.000000,1.000000,1.000000


In [7]:
retail_data_copy.isnull().sum()

Product               0
date                  0
Sales                 0
Price Discount (%)    0
In-Store Promo        0
Catalogue Promo       0
Store End Promo       0
Google_Mobility       0
Covid_Flag            0
V_DAY                 0
EASTER                0
CHRISTMAS             0
dtype: int64

In [8]:
retail_data_copy.duplicated().sum()

0

In [9]:
# Calculate Z-scores to identify outliers
retail_data_copy['Z-Score'] = stats.zscore(retail_data_copy['Sales'])

# Define threshold
threshold = 3

# Identify outliers
outliers = retail_data_copy[abs(retail_data_copy['Z-Score']) > threshold]
print("Outliers based on Z-score:")
outliers

Outliers based on Z-score:


,Product,date,Sales,Price Discount (%),In-Store Promo,Catalogue Promo,Store End Promo,Google_Mobility,Covid_Flag,V_DAY,EASTER,CHRISTMAS,Z-Score
46,SKU1,12/24/2017,155253,17%,1,0,0,0.00,0,0,0,0,3.568388
98,SKU1,12/23/2018,174994,28%,1,0,1,0.00,0,0,0,0,4.132124
150,SKU1,12/22/2019,258874,44%,0,1,1,0.00,0,0,0,0,6.527454
421,SKU3,5/7/2017,212245,50%,1,0,0,0.00,0,0,0,0,5.195887
426,SKU3,6/11/2017,248373,50%,1,0,1,0.00,0,0,0,0,6.227581
437,SKU3,8/27/2017,236242,50%,1,0,1,0.00,0,0,0,0,5.881161
447,SKU3,11/5/2017,212947,50%,1,0,1,0.00,0,0,0,0,5.215934
478,SKU3,6/10/2018,288322,50%,1,0,1,0.00,0,0,0,0,7.368390
494,SKU3,9/30/2018,237462,50%,1,0,1,0.00,0,0,0,0,5.916000
514,SKU3,2/17/2019,195088,50%,1,0,1,0.00,0,0,0,0,4.705941


In [10]:
retail_data_copy = retail_data_copy.drop('Z-Score', axis=1)
retail_data_copy

,Product,date,Sales,Price Discount (%),In-Store Promo,Catalogue Promo,Store End Promo,Google_Mobility,Covid_Flag,V_DAY,EASTER,CHRISTMAS
0,SKU1,2/5/2017,27750,0%,0,0,0,0.00,0,0,0,0
1,SKU1,2/12/2017,29023,0%,1,0,1,0.00,0,1,0,0
2,SKU1,2/19/2017,45630,17%,0,0,0,0.00,0,0,0,0
3,SKU1,2/26/2017,26789,0%,1,0,1,0.00,0,0,0,0
4,SKU1,3/5/2017,41999,17%,0,0,0,0.00,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1213,SKU6,10/18/2020,96619,54%,0,1,0,-7.56,1,0,0,0
1214,SKU6,10/25/2020,115798,52%,0,1,0,-8.39,1,0,0,0
1215,SKU6,11/1/2020,152186,54%,1,0,1,-7.43,1,0,0,0
1216,SKU6,11/8/2020,26445,44%,1,0,1,-5.95,1,0,0,0


In [11]:
retail_data_copy['Another Z-Score'] = stats.zscore(retail_data_copy['Google_Mobility'])
outliers_2 = retail_data_copy[abs(retail_data_copy['Another Z-Score']) > threshold]
print("Outliers based on Z-score:")
outliers_2

Outliers based on Z-score:


,Product,date,Sales,Price Discount (%),In-Store Promo,Catalogue Promo,Store End Promo,Google_Mobility,Covid_Flag,V_DAY,EASTER,CHRISTMAS,Another Z-Score
164,SKU1,3/29/2020,43707,1%,0,0,0,-26.63,1,0,0,0,-4.178667
165,SKU1,4/5/2020,39473,1%,0,1,0,-28.49,1,0,0,0,-4.499141
166,SKU1,4/12/2020,64685,17%,0,0,0,-27.30,1,0,1,0,-4.294106
167,SKU1,4/19/2020,39290,2%,1,0,1,-24.28,1,0,0,0,-3.773767
168,SKU1,4/26/2020,46829,17%,0,0,0,-22.58,1,0,0,0,-3.480861
184,SKU1,8/16/2020,23744,1%,0,0,0,-20.35,1,0,0,0,-3.096637
368,SKU2,3/29/2020,4258,1%,0,0,0,-26.63,1,0,0,0,-4.178667
369,SKU2,4/5/2020,3719,1%,0,0,0,-28.49,1,0,0,0,-4.499141
370,SKU2,4/12/2020,2914,1%,0,0,0,-27.30,1,0,1,0,-4.294106
371,SKU2,4/19/2020,3275,1%,0,0,0,-24.28,1,0,0,0,-3.773767


In [12]:
retail_data_copy = retail_data_copy.drop('Another Z-Score', axis=1)

In [13]:
retail_data_copy.dtypes

Product                object
date                   object
Sales                   int64
Price Discount (%)     object
In-Store Promo          int64
Catalogue Promo         int64
Store End Promo         int64
Google_Mobility       float64
Covid_Flag              int64
V_DAY                   int64
EASTER                  int64
CHRISTMAS               int64
dtype: object

In [14]:
# Convert 'Product' column from object to string
retail_data_copy['Product'] = retail_data_copy['Product'].astype('string')

# Convert 'Date' column from object to datetime
retail_data_copy['date'] = pd.to_datetime(retail_data_copy['date']).dt.date

# Convert 'Price Discount (%)' column from object to float
retail_data_copy['Price Discount (%)'] = retail_data_copy['Price Discount (%)'].replace('%', '', regex=True).astype(float) / 100

In [15]:
retail_data_copy.dtypes

Product               string[python]
date                          object
Sales                          int64
Price Discount (%)           float64
In-Store Promo                 int64
Catalogue Promo                int64
Store End Promo                int64
Google_Mobility              float64
Covid_Flag                     int64
V_DAY                          int64
EASTER                         int64
CHRISTMAS                      int64
dtype: object

In [16]:
retail_data_copy.head()

,Product,date,Sales,Price Discount (%),In-Store Promo,Catalogue Promo,Store End Promo,Google_Mobility,Covid_Flag,V_DAY,EASTER,CHRISTMAS
0,SKU1,2017-02-05,27750,0.00,0,0,0,0.0,0,0,0,0
1,SKU1,2017-02-12,29023,0.00,1,0,1,0.0,0,1,0,0
2,SKU1,2017-02-19,45630,0.17,0,0,0,0.0,0,0,0,0
3,SKU1,2017-02-26,26789,0.00,1,0,1,0.0,0,0,0,0
4,SKU1,2017-03-05,41999,0.17,0,0,0,0.0,0,0,0,0


In [17]:
retail_data_copy.to_csv('retail_data.csv', index=False)